In [4]:
# ruff: noqa
import sys, os
sys.path.append(os.path.abspath("./../feedback-grape"))
sys.path.append(os.path.abspath("./../"))

# ruff: noqa
from helpers import (
    test_implementations,
    calculate_baseline,
)
from tqdm import tqdm
import jax, re
import numpy as np

test_implementations()

jax.config.update("jax_enable_x64", True)

# Evaluation parameters
evaluation_time_steps = 20 # Number of time steps for evaluation
dir_ = "./01_results/"

# Open all filenames in the directory dir_+"/models"
json_files = [f for f in os.listdir(dir_ + "/models") if f.endswith(".json")]
unique_files = []
processed_files = set()
for filename in tqdm(json_files):
    # Skip physically identical files (differing only in sample number)
    s = re.search(r"_s=(\d+)", filename)
    fn = filename.replace(s.group(0), "")
    if fn in processed_files:
        continue
    processed_files.add(fn)
    unique_files.append(filename)
print(f"Found {len(unique_files)} unique model files for evaluation.")
os.makedirs(dir_ + "/baseline", exist_ok=True)

for filename in tqdm(unique_files):
    N_qubits = int(re.search(r"Nqubits=(\d+)_", filename).group(1))
    gamma_z = float(re.search(r"gammaz=([\d.]+)_", filename).group(1))
    gamma_m = float(re.search(r"gammam=([\d.]+)_", filename).group(1))

    fidelities_each, states_each = calculate_baseline(
        N_qubits=N_qubits,
        gamma_z=gamma_z,
        gamma_m=gamma_m,
        evaluation_time_steps=evaluation_time_steps,
    )

    np.savez(f"{dir_}/baseline/Nqubits={N_qubits}_gammaz={gamma_z}_gammam={gamma_m}.npz", fidelities=fidelities_each)

100%|██████████| 5/5 [00:00<00:00, 34952.53it/s]


Found 5 unique model files for evaluation.


100%|██████████| 5/5 [00:03<00:00,  1.41it/s]
